In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import json
import re
from tqdm import tqdm

In [2]:
# GET THE DATASET DIVERSEVUL
f = open('DiverseVul.VulnerableCode.json')
data = json.load(f)
dv = pd.DataFrame(data)
dv.drop(['_id', 'cwe','project','size'], axis='columns', inplace=True)
dv.info()
dv.groupby('target').describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 330492 entries, 0 to 330491
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   func    330492 non-null  object
 1   target  330492 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 5.0+ MB


func                                                                
         count  unique                                                top freq
target                                                                        
0       311547  311547  static int authorizer(void *autharg, int acces...    1
1        18945   18945  static void cirrus_invalidate_region(CirrusVGA...    1

In [3]:
import re
import pandas as pd
from tqdm import tqdm

def remove_comments(code):
    # Rimuovi commenti su una sola riga
    code = re.sub(r'//.*', '', code)
    # Rimuovi commenti su più righe
    code = re.sub(r'/\*.*?\*/', '', code, flags=re.DOTALL)
    return code

def remove_includes_defines(code):
    # Rimuovi include
    code = re.sub(r'#include\s*".*?"', '', code)
    # Rimuovi define
    code = re.sub(r'#define.*', '', code)
    return code

def remove_string(f):
  # f is a string
	# This function removes printed strings: "..."
	f = re.sub("[\"]+(.)+[\"]","\"STR\"",f)
	return f

# Supponiamo che il DataFrame originale si chiami df
df = dv.copy()

# Liste per i nuovi valori
n_tokens, new_strings = [], []

for i in tqdm(range(len(df))):
    file_string = df['func'].iloc[i]
    # Calcola il numero di token prima del preprocessing
    n_tokens.append(len(file_string.split()))
    # Applica le funzioni di preprocessing
    file_string = remove_comments(remove_includes_defines(remove_string(file_string)))
    new_strings.append(file_string)

# Aggiungi le nuove colonne al DataFrame
df['lengths'] = n_tokens
df['func'] = new_strings
df = df[df['lengths']>=5  ]
df = df[df['lengths']<=256]
df.sort_values('lengths')
df = df.drop(['lengths'], axis=1)
df = df.reset_index(drop=True)

df

100%|████████████████████████████████| 330492/330492 [00:07<00:00, 41490.53it/s]


,func,target
0,static void cirrus_invalidate_region(CirrusVGA...,1
1,static int pwc_video_close(struct inode *inode...,1
2,"static int putreg(struct task_struct *child,\n...",1
3,static void cirrus_mem_writeb_mode4and5_16bpp(...,1
4,"static int fat_dir_ioctl(struct inode * inode,...",1
...,...,...
291422,"auth_passdb_preinit(struct auth *auth, const s...",0
291423,"int CMS_data(CMS_ContentInfo *cms, BIO *out, u...",0
291424,static int cms_kari_set1_pkey(CMS_ContentInfo ...,0
291425,static int cms_signerinfo_verify_cert(CMS_Sign...,0


In [6]:
prima_entry_func = dv['func'].iloc[0]

print(f"Prima: {prima_entry_func}")

dopo_entry_func = df['func'].iloc[0]

print(f"Dopo: {dopo_entry_func}")

Prima: static void cirrus_invalidate_region(CirrusVGAState * s, int off_begin,
				     int off_pitch, int bytesperline,
				     int lines)
{
    int y;
    int off_cur;
    int off_cur_end;

    for (y = 0; y < lines; y++) {
	off_cur = off_begin;
	off_cur_end = off_cur + bytesperline;
	off_cur &= TARGET_PAGE_MASK;
	while (off_cur < off_cur_end) {
	    cpu_physical_memory_set_dirty(s->vram_offset + off_cur);
	    off_cur += TARGET_PAGE_SIZE;
	}
	off_begin += off_pitch;
    }
}
Dopo: static int chip_cmd(struct CHIPSTATE *chip, char *name, audiocmd *cmd)
{
	int i;

	if (0 == cmd->count)
		return 0;

	
	v4l_dbg(1, debug, chip->c, "STR",
		chip->c->name, name,cmd->bytes[0]);
	for (i = 1; i < cmd->count; i++) {
		if (debug)
			printk("STR",cmd->bytes[i]);
		chip->shadow.bytes[i+cmd->bytes[0]] = cmd->bytes[i];
	}
	if (debug)
		printk("STR");

	
	if (cmd->count != i2c_master_send(chip->c,cmd->bytes,cmd->count)) {
		v4l_warn(chip->c, "STR", chip->c->name, name);
		return -1;
	}
	return 0;
}


In [4]:
union = df.copy()
vv = df[df['target']==1]
nv = df[df['target']==0].sample(n=len(df[df['target']==1]))
union = pd.concat([vv,nv])
union = union.sample(frac = 1)
union = union.reset_index(drop=True)
union

,func,target
0,"static int chip_cmd(struct CHIPSTATE *chip, ch...",1
1,"unserialize_uep(bufinfo_T *bi, int *error, cha...",1
2,static void describe_balance_args(struct btrfs...,0
3,_nc_set_tputs_trace(const char *s)\n{\n if ...,0
4,PHP_FUNCTION(xmlrpc_set_type)\n{\n\tzval *arg;...,0
...,...,...
25057,static int alarm_timer_nsleep(const clockid_t ...,1
25058,const char* CMSEXPORT cmsIT8GetProperty(cmsHAN...,0
25059,static int kvm_ioctl_create_device(struct kvm ...,1
25060,"char *symbol_string(char *buf, char *end, void...",0


In [5]:
df=union.copy()
# SPLIT IN 3 SET AND STRATITY
df_v = df[df['target']==1]
split1 = int(len(df_v)*0.9)
split2 = int(len(df_v)*0.95)
df_v_train= df_v[:split1]
df_v_valid= df_v[split1:split2]
df_v_test = df_v[split2:]

df_n = df[df['target']==0]
split1 = int(len(df_n)*0.9)
split2 = int(len(df_n)*0.95)
df_n_train= df_n[:split1]
df_n_valid= df_n[split1:split2]
df_n_test = df_n[split2:]

# MERGE AND SHUFFLE ITEMS
df_train = pd.concat([df_v_train,df_n_train])
df_train = df_train.sample(frac = 1)
df_valid = pd.concat([df_v_valid,df_n_valid])
df_valid = df_valid.sample(frac = 1)
df_test  = pd.concat([df_v_test, df_n_test])
df_test  = df_test.sample(frac = 1)

print("TOTAL:",len(df))

print("TRAIN VUL    :",len(df_train[df_train['target']==0]))
print("TRAIN NON VUL:",len(df_train[df_train['target']==1]))
print("VALID VUL    :",len(df_valid[df_valid['target']==0]))
print("VALID NON VUL:",len(df_valid[df_valid['target']==1]))
print("TEST VUL     :",len(df_test[df_test['target']==0]))
print("TEST NON VUL :",len(df_test[df_test['target']==1]))

TOTAL: 25062
TRAIN VUL    : 11277
TRAIN NON VUL: 11277
VALID VUL    : 627
VALID NON VUL: 627
TEST VUL     : 627
TEST NON VUL : 627
